In [70]:
import json
import logging
import mintapi
import os

from typing import Dict

from airflow.hooks.base import BaseHook
from airflow.macros import ds_format
from airflow.models import Variable

from timhealz.common.data_models.spendy import Transaction
from timhealz.spendy import parse_transaction

log = logging.getLogger(__name__)

log.info("Fetching data directory")
DATA_DIR = Variable.get("MINT_DATA_DIRECTORY")
TRANSACTIONS_FP = os.path.join(DATA_DIR, "transactions/{ds}.json")

[2021-12-30 09:21:16,859] {506080088.py:17} INFO - Fetching data directory


In [71]:
log.info("Fetching intuit credentials")
intuit_creds = BaseHook.get_connection("intuit-credentials")

log.info("Initializing Mint session")
mint = mintapi.Mint(
    intuit_creds.login,
    intuit_creds.password,
    headless=True,
    use_chromedriver_on_path=True,
)

[2021-12-30 09:21:20,334] {2277532274.py:1} INFO - Fetching intuit credentials
[2021-12-30 09:21:20,339] {2277532274.py:4} INFO - Initializing Mint session


In [72]:
log.info("Getting transactions")

start_date = ds_format("2021-01-01", "%Y-%m-%d", "%m/%d/%y")
end_date = ds_format("2021-12-28", "%Y-%m-%d", "%m/%d/%y")

data = mint.get_transactions_json(
    start_date=start_date, end_date=end_date
)

[2021-12-30 09:21:38,871] {1128300975.py:1} INFO - Getting transactions


In [73]:
transactions_clean = {}
for i,transaction in enumerate(data):
    try:
        month, day = transaction["date"].split()
        ds = ds_format(f"2021-{month}-{day}","%Y-%b-%d", "%Y-%m-%d")
    except:
        ds = ds_format(transaction["date"], "%m/%d/%y", "%Y-%m-%d")    

    transaction["ds"] = ds

    transactions_clean.setdefault(ds,[]).append(transaction)

In [74]:
for ds, ds_transactions in transactions_clean.items():
    fp = TRANSACTIONS_FP.format(ds=ds)
    
    log.info(f"Dumping transactions json for {ds}")
    with open(fp, "w") as out:
        json.dump(ds_transactions, out, indent=4)


[2021-12-30 09:21:53,306] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-27
[2021-12-30 09:21:53,309] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-28
[2021-12-30 09:21:53,311] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-26
[2021-12-30 09:21:53,312] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-24
[2021-12-30 09:21:53,313] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-23
[2021-12-30 09:21:53,314] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-22
[2021-12-30 09:21:53,316] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-21
[2021-12-30 09:21:53,317] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-20
[2021-12-30 09:21:53,318] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-19
[2021-12-30 09:21:53,328] {2411927436.py:4} INFO - Dumping transactions json for 2021-12-18
[2021-12-30 09:21:53,329] {2411927436.py:4} INFO - Dumping transactions json for